In [1]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
#import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pdb

scipy: 0.19.1
numpy: 1.13.1
matplotlib: 2.0.2
pandas: 0.20.3
sklearn: 0.18.1


C:\Users\SuperBug\Anaconda2\envs\cs_project\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [117]:
with open('../cspython/test.pkl', 'rb') as f: 
     d = pkl.load(f)

In [118]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [4]:
def combine_dfs(big_data, overview):
    for l in range(len(big_data.keys())):
        series_data = big_data[big_data.keys()[l]]
        series_data_m = merge_matches(series_data)
        series_data_m["date"] = overview.loc[l, 'date']
        series_data_mo = merge_overview(series_data_m, series_data)
        series_data_mos = merge_scoreboards(series_data_mo, series_data)
        series_data_mosm = match_data_board_changer(series_data_mos, series_data)
        if l == 0:
            data = series_data_mosm
            
        else:
            col = ['map','round_num','half','match_id','series_id','ending','CT','T','side_winner','winner','team_A','team_B','team_A_score','team_B_score','match_num','team_players','K-D','+/-','ADR','KAST','Rating2.0','nicknames']
            data = pd.concat([data, series_data_mosm])  
            col2 = data.columns
            data = data[col + list(set(col2) - set(col))]
    data = data.reset_index()
    return data

In [5]:
def merge_matches(series_data):
    for d in range(0,len(series_data['matches'])):
        if d == 0:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = series_data['matches'][d]
        else:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = pd.concat([series_data_m, series_data['matches'][d]])
    
    return series_data_m
    #should work for concact the matches together


In [6]:
def match_data_board_changer(series_data_mos,series_data):
    
    board_name = ['first_kills','who_kill_who', 'awp_kills']

    for idx, a in enumerate(series_data['match_data']):
        new_df = pd.DataFrame()
        for idx1, c in enumerate(board_name):    
            new_board_c = pd.DataFrame()
            new_board_r= pd.DataFrame()
            names_c = a[c].set_index('Unnamed: 0').columns
            for b in names_c:
                new_board_c[b+'_'+c] = a[c].set_index('Unnamed: 0')[b].str.split(pat = ':', expand = True)[0]
               
            names_r = a[c].set_index('Unnamed: 0').T.columns
            for b in names_r:
                new_board_r[b+'_'+c] = a[c].set_index('Unnamed: 0').T[b].str.split(pat = ':', expand = True)[1]
            new_board_c['nicknames'] = new_board_c.index
            new_board_r['nicknames'] = new_board_r.index
            
            board_df = new_board_c.append(new_board_r)
            
            if idx1 == 0:
                new_df = board_df
            else:
                new_df = pd.concat([new_df, board_df], axis = 1)
        if idx == 0:
            new_df['match_num'] = 1+idx
            con_df = new_df
        else:
            new_df['match_num'] = 1+idx
            con_df = con_df.append(new_df, ignore_index=True)
    con_df = con_df.loc[:, ~con_df.columns.duplicated()]
    series_data_mosm = pd.merge(series_data_mos, con_df, on=['nicknames', 'match_num'], how = 'inner')
    return series_data_mosm


In [7]:
def merge_overview(series_data_m, series_data):
    series_data['match_overview']['team_A_name'] = series_data['match_overview'].columns[4]
    series_data['match_overview']['team_B_name'] = series_data['match_overview'].columns[5]
    
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] == series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_B_name
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] != series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_A_name
    series_data['match_overview'] = series_data['match_overview'].rename(index = str, columns ={series_data['match_overview'].columns[3]: "winner_of_match",series_data['match_overview'].columns[4]: "team_A_score",series_data['match_overview'].columns[5]: "team_B_score"})
    
    
    series_data_mo = pd.merge(series_data_m, series_data['match_overview'], on=['match_id', 'map', 'series_id'])
    
    return series_data_mo
#works at merging matches with match_overviewb

In [8]:
def merge_scoreboards(series_data_mo, series_data):    
   
    for i in range(len(series_data['scoreboards'][0])):
        series_data['scoreboards'][0][i]['match_num'] = i+1
        #pdb.set_trace()
        series_data['scoreboards'][0][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_A_name'].unique()[0]
        series_data['scoreboards'][0][i] = series_data['scoreboards'][0][i].rename(index = str, columns={ series_data['scoreboards'][0][i].columns[0] : "team_players" })
        
        series_data['scoreboards'][1][i]['match_num'] = i+1
        series_data['scoreboards'][1][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_B_name'].unique()[0]
        series_data['scoreboards'][1][i] = series_data['scoreboards'][1][i].rename(index = str, columns={ series_data['scoreboards'][1][i].columns[0] : "team_players"})
        
        new_df = pd.concat([series_data['scoreboards'][0][i], series_data['scoreboards'][1][i]])
        
        if i == 0:
            con_df = new_df
        else:
            con_df = pd.concat([con_df, new_df])
   
    series_data_mos = pd.merge(series_data_mo, con_df, how='outer', on='match_num')
    series_data_mos['nicknames'] = series_data_mos['team_players'].str.split(expand = True)[3]
    return series_data_mos

       
        
#works at adding match_num to scoreboards
        
                         

In [119]:
data = combine_dfs(big_data, overview)

In [10]:
def create_fwa_dr_columns(data, col_list):  # first k , awp, who, divided by rounds 
    columns = pd.Series(data.columns)
    for a in col_list:
        col = columns[columns.str.contains(a)]
        data[a+'_sum_dr'] = data[col].convert_objects(convert_numeric = True).sum(axis = 1) / (data['team_A_score'] + data['team_B_score'] - 5)
    data = data.apply(pd.to_numeric, errors='ignore')
    return data
        
    

In [11]:
def create_fwadr_his(data, col_list):   # column with historic awp, first ,who_killwho of player vs player.
    match_id = data.match_id.unique()
    players = data.nicknames.unique()
    for a in col_list:
        data[a + '_sum_dr_hist'] = np.nan
        for b in match_id:
            for c in players:
                values = data.loc[(data['match_id'] == b)&(data['nicknames'] == c),a +'_sum_dr'].unique()
                data.loc[(data['match_id'] == b)&(data['nicknames'] == c),a + '_sum_dr_hist'] = values.sum()/ float(len(values))
    return data
    

In [12]:
def create_avdamage_his(data):  # column with historic average damage of individual ( NEEDS TO BE FOR EACH MAP)
    match_id = data.match_id.unique()
    players = data.nicknames.unique()
    data['ADR_hist'] = nan
    for b in data.match_id.unique():
            for c in players:
                values = data.loc[(data['match_id'] == b)&(data['nicknames'] == c),'ADR'].unique()
                data.loc[(data['match_id'] == b)&(data['nicknames'] == c),'ADR_hist'] = values.sum()/ float(len(values))
    return data
    

In [13]:
# total of times  player played each map

In [14]:
# total of times player won on each map

In [15]:
# percentage of times player on on map

In [16]:
def create_map_win_loss_his(data):  # team total win and loses on map with total times played on map
    maps = data.map.unique()
    teams = data.player_team_name.unique()
    match_id = data.match_id.unique()
    for a in maps:
        data[a + "_win_his"] = 0
        data[a + "_loss_his"] = 0
        data[a + "_total_played"] = 0
    for a in match_id:
        map_for_match = data.loc[(data['match_id'] == a) ,'map'].unique()
        winner_of_map = data.loc[(data['match_id'] == a), 'winner_of_match'].unique()
        loser_of_map = data.loc[(data['match_id'] == a), 'loser_of_match'].unique()
        data.loc[(data['player_team_name'] == winner_of_map[0]), [map_for_match[0] + "_win_his", map_for_match[0] +'_total_played']] += 1
        data.loc[(data['player_team_name'] == loser_of_map[0]), [map_for_match[0] + "_loss_his", map_for_match[0] +'_total_played']] += 1 
       
    return data

In [17]:
def create_map_win_his_per(data): #percentage team total win and loses on map
    teams = data.player_team_name.unique()
    maps = data.map.unique()
    for a in maps:
        data[a + '_win_perc_map'] = 0
        for b in teams:
            pg = (data.player_team_name == b) 
            data.loc[pg,a + '_win_perc_map'] = data.loc[pg, a + "_win_his"].unique()[0] / (data.loc[pg, a + "_win_his"].unique()[0] + data.loc[pg, a + "_loss_his"].unique()[0]) 
            data.loc
    return data    
    
    

In [124]:
def create_rounds_won_vs_team_his(data): # team rounds won vs another team
    grouping = data.groupby(['match_id','map','team_A_name','team_B_name', 'team_A_score', 'team_B_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['team_A_name', 'team_B_name']).sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    forward = grouping.team_A_name+grouping.team_B_name
    reverse = grouping.team_B_name+grouping.team_A_name
    for idx, val in enumerate(forward):
        for idx2, val2 in enumerate(reverse):
            if val == val2:
                grouping.loc[val,'team_A_score_Count'] += grouping.loc[val2,'team_B_score_Count']
                grouping.loc[val,'team_B_score_Count'] += grouping.loc[val2,'team_A_score_Count']
                grouping.loc[val2,'team_B_score_Count'] = grouping.loc[val,'team_A_score_Count']
                grouping.loc[val2,'team_A_score_Count'] = grouping.loc[val,'team_B_score_Count']
    
    grouping = grouping.drop('round_num_Count_Count', axis = 1)
    col1 = list(grouping.team_A_name.unique())
    col2  = list(grouping.team_B_name.unique())
    col = col1 + col2
    col = list(set(col))
    data = pd.merge(data,grouping, on=['team_A_name', 'team_B_name']) 
    for a in col:
        data['rd_total_his_'+ a] = 0
        data.loc[(data.player_team_name != a) & (data.team_A_name == a) , 'rd_total_his_'+ a]=data.team_B_score_Count
        data.loc[(data.player_team_name != a) & (data.team_B_name == a) , 'rd_total_his_'+ a]=data.team_A_score_Count
    
    return data    

In [125]:
def create_total_team_rd_map_his(data):
    grouping = data.groupby(['map','team_A_name', 'team_A_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['map','team_A_name'])[ 'team_A_score'].sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    fgrouping = data.groupby(['player_team_name','map','team_B_name', 'team_B_score'])['round_num'].count()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.groupby(['map','team_B_name'])[ 'team_B_score'].sum()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.rename(index=str, columns={"team_B_name": "team_A_name", 'team_B_score_Count': 'team_A_score_Count'})
    merged = pd.concat([grouping, fgrouping], axis = 0)
    merged.groupby(['map', 'team_A_name'])['team_A_score_Count'].sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index(drop = True)
    merged = merged.rename(index=str, columns={"team_A_name": "player_team_name", "team_A_score_Count": 'total_team_rd_map'})
    merged = merged.groupby(['player_team_name', 'map']).sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index()
    for a in list(merged.map.unique()):
        merged.loc[:,'total_team_rd_'+ a] = 0
        merged.loc[(merged.loc[:, 'map'] == a), 'total_team_rd_'+ a] = merged.loc[:,'total_team_rd_map']
    merged = merged.drop('total_team_rd_map', axis = 1)
    data = pd.merge(data, merged, on =['player_team_name', 'map'])
    return data
    

In [122]:
col_list = ['_first_kills', '_who_kill_who','_awp_kills']
data = create_fwa_dr_columns(data, col_list)
data = create_fwadr_his(data,col_list)
data = create_avdamage_his(data)
data = create_map_win_loss_his(data)
data = create_map_win_his_per(data)
data = create_rounds_won_vs_team_his(data)
data = create_total_team_rd_map_his(data)

In [127]:
for a in data.columns:
    print a

index
map
round_num
half
match_id
series_id
ending
CT
T
side_winner
winner
team_A
team_B
team_A_score
team_B_score
match_num
team_players
K-D
+/-
ADR
KAST
Rating2.0
nicknames
GuardiaN_who_kill_who
kabal_first_kills
BlackEagle_who_kill_who
dobu_first_kills
ryx_who_kill_who
vRss_awp_kills
coldzera_awp_kills
Stewie2K_awp_kills
dispenser_who_kill_who
apEX_first_kills
xiaosaGe_awp_kills
shouta_who_kill_who
cadiaN_who_kill_who
forget_first_kills
Empera_who_kill_who
JustnatioN_who_kill_who
flusha_awp_kills
Lekr0_who_kill_who
yay_first_kills
Skadoodle_first_kills
Edward_awp_kills
player_team_name
PTC_who_kill_who
Zeus_who_kill_who
cheK1vl_awp_kills
Blue +_who_kill_who
vRss_who_kill_who
dephh_awp_kills
Fable_who_kill_who
originalheart_first_kills
bramz_who_kill_who
electronic_awp_kills
winner_of_match
shanx_awp_kills
BlackEagle_first_kills
felps_first_kills
GuardiaN_first_kills
TACO_first_kills
originalheart_awp_kills
KRIMZ_who_kill_who
Potei_who_kill_who
hoveRR_awp_kills
felps_who_kill_who
sho

In [126]:
data.groupby('nicknames', '_first_kills')

ValueError: No axis named _first_kills for object type <class 'pandas.core.frame.DataFrame'>

In [ ]:
merged.loc(:)

In [511]:
pd.set_option('display.max_columns', 300)

In [ ]:
#historical percentage of win on the map
#historical rounds for rounds against vs team
#historical rounds for on map
historical average of first picks per person  / on map divided by rounds
historical average of amount of kill per person/ on map divided by roundfor 
historical average of amount of awp kills per person / on map divided by rounds
#historical average of damage done / on map 
#historical average of first picks / on map divided by rounds
#historical average of awp kills / on map divided by rounds
#historical average of kills / on map divided by rounds
previous 3 match average kills  divided by rounds
previous 3 match average first picks divided by rounds
previous 3 match average awp kills divided by rounds
previous 3 match average damage 






In [14]:
for a in data.columns:
    if Series.str.contains
    print a

+/-
ADR
ANDROID-X23_awp_kills
ANDROID-X23_first_kills
ANDROID-X23_who_kill_who
BlackEagle_awp_kills
BlackEagle_first_kills
BlackEagle_who_kill_who
Blue +_awp_kills
Blue +_first_kills
Blue +_who_kill_who
CT
Edward_awp_kills
Edward_first_kills
Edward_who_kill_who
Empera_awp_kills
Empera_first_kills
Empera_who_kill_who
FNS_awp_kills
FNS_first_kills
FNS_who_kill_who
Fable_awp_kills
Fable_first_kills
Fable_who_kill_who
FalleN_awp_kills
FalleN_first_kills
FalleN_who_kill_who
Fog_awp_kills
Fog_first_kills
Fog_who_kill_who
Golden_awp_kills
Golden_first_kills
Golden_who_kill_who
GuardiaN_awp_kills
GuardiaN_first_kills
GuardiaN_who_kill_who
Hiko_awp_kills
Hiko_first_kills
Hiko_who_kill_who
JW_awp_kills
JW_first_kills
JW_who_kill_who
Jada_awp_kills
Jada_first_kills
Jada_who_kill_who
JustnatioN_awp_kills
JustnatioN_first_kills
JustnatioN_who_kill_who
K-D
KAST
KRIMZ_awp_kills
KRIMZ_first_kills
KRIMZ_who_kill_who
Kheops_awp_kills
Kheops_first_kills
Kheops_who_kill_who
Lekr0_awp_kills
Lekr0_first_kil

In [922]:
data.shape

(2831, 247)

In [50]:
data.tea

AttributeError: 'DataFrame' object has no attribute 'team_name'

In [13]:
pd.set_option('display.max_columns', None)

In [31]:
name_search = data.loc[data['round_num'] == 1]
name_search_sum = name_search.loc[name_search['nicknames'] == 'FalleN'].convert_objects(convert_numeric = True).sum()


In [34]:
name_search_sum[name_search_sum > 0].sort_values()

Lekr0_first_kills                                                      1
flusha_first_kills                                                     1
team_B                                                                 1
JW_first_kills                                                         2
team_A                                                                 2
round_num                                                              3
half                                                                   3
Rating2.0                                                           3.32
KRIMZ_first_kills                                                      4
Golden_first_kills                                                     4
KRIMZ_awp_kills                                                        5
+/-                                                                    6
Golden_awp_kills                                                       6
match_num                                          

In [26]:
data.loc[data['round_num'] == 1].convert_objects(convert_numeric = True).sum()

+/-                                                                         8
ADR                                                                    8608.4
ANDROID-X23_awp_kills                                                       3
ANDROID-X23_first_kills                                                     2
ANDROID-X23_who_kill_who                                                   21
BlackEagle_awp_kills                                                        2
BlackEagle_first_kills                                                     10
BlackEagle_who_kill_who                                                    36
Blue +_awp_kills                                                            3
Blue +_first_kills                                                          5
Blue +_who_kill_who                                                        28
CT                          LimitlessLimitlessLimitlessLimitlessLimitlessL...
Edward_awp_kills                                                

In [ ]:
data = pd.concat([data,data['K-D'].str.split('-', 1, expand=True).rename(columns={0:'player_kill', 1:'player_death'})], axis=1)


In [ ]:
data.shape

In [ ]:
data = data.apply(pd.to_numeric, errors='ignore')

In [ ]:
data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']]= data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']].apply(lambda x: pd.factorize(x)[0]) 

In [ ]:
data['KAST'] = data['KAST'].apply(lambda x : float(x.strip('%')) / 100.0)

In [ ]:
data.dtypes

In [ ]:
corr = data.select_dtypes(include = ['float64', 'int64', 'int32']).iloc[:, 0:].corr()
plt.figure(figsize=(25, 25))
sns.heatmap(corr, vmax=1, square=True)



In [ ]:
ydata = data['winner']

In [ ]:
data = data.drop('winner', 1)

In [ ]:
data = data.drop('match_id', 1)

In [ ]:
data = data.drop('series_id', 1)

In [ ]:
data = data.drop('K-D', 1)

In [ ]:
data.shape

In [ ]:
y = ydata.values
X = data.values[:,0:19]

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'

In [ ]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'
model = GradientBoostingClassifier()
params = {
 'max_depth':[4,6,8,10,12,16],
 'n_estimators':[20,30,50,80,100],
'learning_rate':[.1,.01,.001]}
local_cv(model, params)

In [ ]:
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('NB', GaussianNB()))
#models.append(('CART', DecisionTreeClassifier())) 
#models.append(('KNeighborsClassifier', KNeighborsClassifier())) 
#models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
#models.append(('AdaBoostClassifier', AdaBoostClassifier()))
#models.append(('RandomForestClassifier', RandomForestClassifier()))
#models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('SVC', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    print '1'
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    print '2'
    cv_results = cross_validation.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    print '3'
    results.append(cv_results)
    print '4'
    names.append(name)
    print '5'
    model.fit(X, y)
    print '6'
    predicted = model.predict(X)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)